# Creating an Lennard Jones fluid in OpenMM

## Preliminaries

First, we import OpenMM. It's recommended that you always import this way:

In [2]:
from simtk import openmm, unit
from simtk.openmm import app
import numpy as np


## The OpenMM `System` object

The OpenMM [`System`](http://docs.openmm.org/7.1.0/api-python/library.html#system) object is a container that completely specifies how to compute the forces and energies for all particles in the molecular system. 

Because it is part of the C++ OpenMM API, `System` (and the related `Force` classes) are not true Python objects but instead SWIG-wrapped C++ objects, so you will find their naming and accessor conventions conform to C++ (rather than Pythonic) standards.

There are many ways to create or import OpenMM `System` objects:
* We can assemble all the particles and define their interactions ourselves programmatically using the OpenMM C++ API
* We could load in a serialized version of the `System` object via the built-in XML serializer
* We can use the various importers in the [`simtk.openmm.app`](http://docs.openmm.org/7.1.0/api-python/app.html) layer to read in a system definition in AMBER, CHARMM, or gromacs format
* We can create our own `System` object from a PDB file using the OpenMM [`ForceField`](http://docs.openmm.org/7.1.0/userguide/application.html#force-fields) facility
* We can use a large variety of pre-built test systems provided by the [`openmmtools.testsystems`](http://openmmtools.readthedocs.io) module for testing code on a battery of different system types with different kinds of forces

Remember that if you want more information on `openmm.System`, you can use `help(openmm.System)`:

## Creating a periodic Lennard-Jones system

We'll start by building a simple periodic Lennard-Jones system using the Python wrappers for the main OpenMM C++ API.

In [3]:
# Define the parameters of the Lennard-Jones periodic system
nparticles = 512
reduced_density = 0.05
mass = 39.9 * unit.amu
charge = 0.0 * unit.elementary_charge
sigma = 3.4 * unit.angstroms
epsilon = 0.238 * unit.kilocalories_per_mole

# Create a system and add particles to it
system = openmm.System()
for index in range(nparticles):
    # Particles are added one at a time
    # Their indices in the System will correspond with their indices in the Force objects we will add later
    system.addParticle(mass)
    
# Set the periodic box vectors
number_density = reduced_density / sigma**3
volume = nparticles * (number_density ** -1)
box_edge = volume ** (1. / 3.)
box_vectors = np.diag([box_edge/unit.angstrom for i in range(3)]) * unit.angstroms
system.setDefaultPeriodicBoxVectors(*box_vectors)

# Add Lennard-Jones interactions using a NonbondedForce
force = openmm.NonbondedForce()
force.setNonbondedMethod(openmm.NonbondedForce.CutoffPeriodic)
for index in range(nparticles): # all particles must have parameters assigned for the NonbondedForce
    # Particles are assigned properties in the same order as they appear in the System object
    force.addParticle(charge, sigma, epsilon)
force.setCutoffDistance(3.0 * sigma) # set cutoff (truncation) distance at 3*sigma
force.setUseSwitchingFunction(True) # use a smooth switching function to avoid force discontinuities at cutoff
force.setSwitchingDistance(2.5 * sigma) # turn on switch at 2.5*sigma
force.setUseDispersionCorrection(True) # use long-range isotropic dispersion correction
force_index = system.addForce(force) # system takes ownership of the NonbondedForce object

### The OpenMM Python API uses units throughout

Note the use of unit-bearing quantities via [`simtk.unit`](http://docs.openmm.org/7.1.0/userguide/library.html#units-and-dimensional-analysis). OpenMM's Python API uses a powerful units system that prevents many common kinds of mistakes with unit conversion, such as those that caused the loss of the [Mars Polar Lander](http://www.cnn.com/TECH/space/9909/30/mars.metric.02/). We'll examine the units system in more detail later---for now, we simply recognize that best practices dictate we feed OpenMM unit-bearing quantities (though this is not strictly necessary), and that it will return unit-bearing quantities.

In [4]:
# construct a Quantity using the multiply operator
bond_length = 1.53 * unit.nanometer
# equivalently using the explicit Quantity constructor
bond_length = unit.Quantity(1.53, unit.nanometer)
# or more verbosely
bond_length = unit.Quantity(value=1.53, unit=unit.nanometer)
# print a unit in human-readable form
print(bond_length)
# or display it in machine-readable form
repr(bond_length)

1.53 nm


'Quantity(value=1.53, unit=nanometer)'

## Accesssors for System attributes and Forces

The [`System`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.System.html#simtk.openmm.openmm.System) object has a number of useful accessors for examining the contents of the system and 

In [5]:
# Get the number of particles in the System
print('The system has %d particles' % system.getNumParticles())
# Print a few particle masses
for index in range(5):
    print('Particle %5d has mass %12.3f amu' % (index, system.getParticleMass(index) / unit.amu))
# Print number of constraints
print('The system has %d constraints' % system.getNumConstraints())
# Get the number of forces and iterate through them
print('There are %d forces' % system.getNumForces())
for (index, force) in enumerate(system.getForces()):
    print('Force %5d : %s' % (index, force.__class__.__name__))

The system has 512 particles
Particle     0 has mass       39.900 amu
Particle     1 has mass       39.900 amu
Particle     2 has mass       39.900 amu
Particle     3 has mass       39.900 amu
Particle     4 has mass       39.900 amu
The system has 0 constraints
There are 1 forces
Force     0 : NonbondedForce


### Define positions for the Lennard-Jones particles

To compute energies or forces, we will need to define some positions for the system before we can compute energies or forces.
We'll generate some randomly, but as we will see later, there are many more ways to load molecular positions.

In [6]:
positions = box_edge * np.random.rand(nparticles,3) 
print(positions)

[[ 10.02668162  39.3175152   56.80219223]
 [  3.88575932  38.36109416  22.11328491]
 [ 58.63268614  40.30017222  55.50949334]
 ..., 
 [  3.22883112  23.76743759   9.14358205]
 [ 43.61167517  54.66140001  34.06724003]
 [ 58.69035833  65.81574166  11.156523  ]] A


## Computing energies and forces

To compute energies and forces efficiently, OpenMM requires you create a [`Context`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Context.html#simtk.openmm.openmm.Context) that handles the computation on a particular piece of hardware. This could be a GPU or CPU using a specific set of fast computation kernels.

OpenMM forces us to explicitly move data (positions, velocities, forces) into and out of the `Context` through specific calls so that we remember that each of these operations carries overhead of moving data across the bus to the GPU, for example. To achieve maximum speed, OpenMM can perform a number of operations---such as integrating many molecular dynamics steps---fully on the GPU without slowing things down by moving data back and forth over the bus. We'll see examples of this soon.

First, we just want to compute energies and forces and then minimize the energy of our system. To do that, we first need to create an [`Integrator`](http://docs.openmm.org/7.1.0/userguide/application.html#integrators) that will be bound to the [`Context`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Context.html#simtk.openmm.openmm.Context):

In [7]:
# Create an integrator
timestep = 1.0 * unit.femtoseconds
integrator = openmm.VerletIntegrator(timestep)
# Create a Context using the default platform (the fastest abailable one is picked automatically)
# NOTE: The integrator is bound irrevocably to the context, so we can't reuse it
context = openmm.Context(system, integrator)

The `System` object can be used by multiple `Contexts` because the `System` is copied and converted to the efficient `Platform` specific kernels inside the `Context` object. Changing parameters or adding new `Force`'s on the Python `System` itself will not update the representation inside the `Context`.

In [8]:
from copy import deepcopy
# Create a new integrator we can throw away
throwaway_integrator = openmm.VerletIntegrator(timestep)
# Copy the System object for this example
copy_system = deepcopy(system)
throwaway_context = openmm.Context(copy_system, throwaway_integrator)
# Change something in the Python System, e.g. double the box size
copy_system.setDefaultPeriodicBoxVectors(*box_vectors*2)
# Observe how the box size in the Context and Python System are different
print([vector for i, vector in enumerate(copy_system.getDefaultPeriodicBoxVectors())])
print([vector for i, vector in enumerate(throwaway_context.getState().getPeriodicBoxVectors())])
# Cleanup
del throwaway_integrator, throwaway_context

[Quantity(value=(14.766431834276288, 0.0, 0.0), unit=nanometer), Quantity(value=(0.0, 14.766431834276288, 0.0), unit=nanometer), Quantity(value=(0.0, 0.0, 14.766431834276288), unit=nanometer)]
[Quantity(value=(7.383215917138144, 0.0, 0.0), unit=nanometer), Quantity(value=(0.0, 7.383215917138144, 0.0), unit=nanometer), Quantity(value=(0.0, 0.0, 7.383215917138144), unit=nanometer)]


There are ways to update paramters in an existing `Context`, but that topic will be covered in later tutorials.

Once we have a [`Context`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Context.html#simtk.openmm.openmm.Context), we need to tell OpenMM we want to retrieve the energy and forces for a specific set of particle positions. This is done by retrieving a [`State`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.State.html#simtk.openmm.openmm.State) object from the [`Context`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Context.html#simtk.openmm.openmm.Context) that contains *only* the information we want to retrieve so as to minimize the amount of data that needs to be sent over the bus from a GPU:

In [9]:
# Set the positions
context.setPositions(positions)
# Retrieve the energy and forces
state = context.getState(getEnergy=True, getForces=True)
potential_energy = state.getPotentialEnergy()
print('Potential energy: %s' % potential_energy)
forces = state.getForces()
print('Forces: %s' % forces[0:5])

Potential energy: 25481395.44935387 kJ/mol
Forces: [(0.0855492502450943, 0.27713125944137573, -0.03837815672159195), (-0.06485646218061447, -0.0715872049331665, -0.033643413335084915), (-0.11481528729200363, 0.07818984985351562, -0.051977600902318954), (0.2687129080295563, 0.5225710272789001, -0.8533488512039185), (-0.23557403683662415, 0.45656919479370117, -0.33631595969200134)] kJ/(nm mol)


Note that the forces are returned as a `list` of tuples with units attached. If we want a numpy array instead, we could convert it, but it's much easier just use the `asNumpy=True` argument to [`state.getForces`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.State.html#simtk.openmm.openmm.State.getForces)

In [10]:
# Get forces as numpy array
forces = state.getForces(asNumpy=True)
print('Forces: %s' % forces)

Forces: [[  8.55492502e-02   2.77131259e-01  -3.83781567e-02]
 [ -6.48564622e-02  -7.15872049e-02  -3.36434133e-02]
 [ -1.14815287e-01   7.81898499e-02  -5.19776009e-02]
 ..., 
 [  1.05605495e+00  -2.54847836e+00  -1.71441197e+00]
 [  3.29639763e-01   9.51404274e-02   4.19779569e-02]
 [ -9.03373489e+01  -2.44110718e+02   2.29076157e+02]] kJ/(nm mol)


## Minimizing the potential energy

You'll notice the energy and force magnitude is pretty large:

In [11]:
# Compute the energy
state = context.getState(getEnergy=True, getForces=True)
potential_energy = state.getPotentialEnergy()
print('Potential energy: %s' % potential_energy)
# Compute the force magnitude
forces = state.getForces(asNumpy=True)
force_magnitude = (forces**2).sum().sqrt()
print('Force magnitude: %s' % force_magnitude)

Potential energy: 25481395.44935387 kJ/mol
Force magnitude: 2601951202.859449 kJ/(nm mol)


If we tried to do dynamics at this point, the system will likely explode!

OpenMM provides a [`LocalEnergyMinimizer`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.LocalEnergyMinimizer.html#simtk.openmm.openmm.LocalEnergyMinimizer) that can be used to bring the force magnitude to be low enough to be able to run a simulation that won't explode.

In [12]:
# Minimize the potential energy
openmm.LocalEnergyMinimizer.minimize(context)

# Compute the energy and force magnitude after minimization
state = context.getState(getEnergy=True, getForces=True)
potential_energy = state.getPotentialEnergy()
print('Potential energy: %s' % potential_energy)
forces = state.getForces(asNumpy=True)
force_magnitude = (forces**2).sum().sqrt()
print('Force magnitude: %s' % force_magnitude)

Potential energy: -187.03488039219292 kJ/mol
Force magnitude: 131.17563166519946 kJ/(nm mol)


Now we want to visualize the system.

In [13]:
from openmmtools import testsystems
t = testsystems.LennardJonesFluid(reduced_density=0.02)
def visualize(t):
    import mdtraj
    mdtraj_topology = mdtraj.Topology.from_openmm(t.topology)
    traj = mdtraj.Trajectory([t.positions/unit.nanometers], mdtraj_topology)

    # View it in nglview
    import nglview
    view = nglview.show_mdtraj(traj)
    if t.system.getNumParticles() < 2000:
        view.add_representation('spacefill','all')
    view.center_view()
    return view

visualize(t)

NGLWidget()